In [ ]:
from tensorflow import keras as keras
import tensorflow as tf
import csv
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# imports needed for CNN
import csv
import cv2
import os, glob
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import time
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from matplotlib import pyplot as plt


In [ ]:
!ls ../input/corners

In [ ]:
IMAGE_WIDTH = 255
IMAGE_HEIGHT = 255

In [ ]:
data = pd.read_csv('../input/corners/data.csv')
data.head()

In [ ]:
def load_data(data_dir):
    images = []
    labels = []
    
    for i in range(len(data)):
        img = cv2.imread(data_dir+data['loc'][i])
        imresize = cv2.resize(img, (255, 255))
        images.append(imresize)
        labels.append([data['x'][i], data['y'][i]])
    
    return images, labels

In [ ]:
data_dir = "../input/corners/ims/"
images, labels = load_data(data_dir)

In [ ]:
print(images[0:10])
print(labels[0:10])

In [ ]:
def cross_validate(Xs, ys):
    X_train, X_test, y_train, y_test = train_test_split(
            Xs, ys, test_size=0.2, random_state=0)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = cross_validate(images, labels)

In [ ]:
X_train = np.array(X_train).astype('float32')
X_test = np.array(X_test).astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
print(X_train.size, X_test.size, y_train.size, y_test.size)

In [ ]:
print(len(y_train))

In [ ]:
print(len(X_train))

In [ ]:
# Configure Model
# model = keras.Sequential([keras.layers.Flatten(input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3)),
#                          keras.layers.Dense(128, activation="relu"),
#                          keras.layers.Dropout(0.1),
#                          keras.layers.Dense(64, activation="relu"),
#                          keras.layers.Dense(2)
#                          ])
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(255, 255, 3), border_mode='same', activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same', W_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dense(2))

epochs = 3  # >>> should be 25+
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
# model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
# # Compile model
model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='mse',
              metrics=['mae'])

print(model.)

In [ ]:
# Train model
# model.fit(Xtrain, Ytrain, epochs=500)
model.fit(X_train, [y_train], validation_data=(X_test, [y_test]), nb_epoch=epochs, batch_size=64)

In [ ]:
# Load test data
def load_testset():
    Xtest = []
    with open('../input/test/test.csv') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            img = np.zeros((IMAGE_HEIGHT,IMAGE_WIDTH,1), dtype=np.float)
            for i, val in enumerate(row["Image"].split(" ")):
                img[i//IMAGE_WIDTH,i%IMAGE_WIDTH,0] = val
            Xtest.append(img)
                
    return np.array(Xtest)
Xtest = load_testset()

In [ ]:
# Preview results on test data
def show_results(image_index):
    Ypred = model.predict(Xtest[image_index:(image_index+1)])
    show_image(Xtest[image_index], Ypred[0])

In [ ]:
show_results(3)

In [ ]:
show_results(4)

In [ ]:
show_results(5)

In [ ]:
def load_data(data_dir):
    """
    From: https://medium.com/@waleedka/traffic-sign-recognition-with-tensorflow-629dffc391a6#.v471kaepx
    """
    # Get all subdirectories of data_dir. Each represents a label.
    directories = [d for d in os.listdir(data_dir)
                   if os.path.isdir(os.path.join(data_dir, d))]
    # Loop through the label directories and collect the data in
    # two lists, labels and images.
    labels = []
    images = []

    category = 0
    for d in directories:
        label_dir = os.path.join(data_dir, d)
        file_names = [os.path.join(label_dir, f)
                      for f in os.listdir(label_dir)
                      if f.endswith(".jpg")]
        
        # adding an early stop for sake of speed
        stop = 0
        for f in file_names:
            img = cv2.imread(f)
            imresize = cv2.resize(img, (200, 125))
            #plt.imshow(imresize)
            images.append(imresize)
            labels.append(category)
            # remove this to use full data set
            if stop > 30:
                break
            stop += 1
            # end early stop
            
        category += 1

    return images, labels

data_dir = "../input"
images, labels = load_data(data_dir)

# confirm that we have the data
print(images[0:10])
print(labels[0:10])